In [1]:
import Data_Conversion
import Input_Parameters 
import Data_Conversion
import Passive_Model
import Solar_Generation
import Electrical_Load
import SO

/Users/jimmy/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import numpy as np
import pandas as pd

In [3]:
# Sensitivity Analysis on Locations, Stochastic Optimization
# Fred Fan, Stanford University, 03/17/2025

In [4]:
# Define the base data directory, list of locations, and the weather year.
data_dir = "Data"
locations = ["Arizona", 
             "Alaska", 
             "Minnesota", 
             "Florida", 
             "HalfMoonBay"]

# 20 years of training data
weather_year_list_training = list(range(1998, 2018))  # use list(range(1998, 2018)) for full dataset
# 5 years of training data
weather_year_list_testing = list(range(2018, 2023))  # use list(range(2018, 2023)) for full dataset

# Set a random seed for consistency
# Define the number of rows (i) and columns (j)
i = len(locations)  # Number of rows
j = len(weather_year_list_training) + len(weather_year_list_testing)  # Number of columns

# Generate a sequential array of numbers starting from 1
total_cells = i * j  # Total number of cells in the DataFrame
sequential_numbers = np.arange(1, total_cells + 1)  # Generates 1, 2, 3, ..., i*j

# Reshape the array into a 2D array with i rows and j columns
data = sequential_numbers.reshape(i, j)

# Convert the array into a DataFrame
random_seeds = pd.DataFrame(data, columns=[f'Column_{k+1}' for k in range(j)])

lats, lons, timezones = Data_Conversion.get_timezones(data_dir, locations)

Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_1998.csv
Reading file for Alaska: Data/NREL_NSRDB_Alaska/34776_59.25_-154.62_1998.csv
Reading file for Minnesota: Data/NREL_NSRDB_Minnesota/777900_44.97_-93.26_1998.csv
Reading file for Florida: Data/NREL_NSRDB_Florida/1060499_25.77_-80.18_1998.csv
Reading file for HalfMoonBay: Data/NREL_NSRDB_HalfMoonBay/137344_37.49_-122.42_1998.csv


In [5]:
# Create dictionary for recording results
training_results = {}  # Initialize as an empty dictionary

In [6]:
len(weather_year_list_training)

20

In [7]:
# Sensitivity Analysis on Locations

for i in range(len(locations)):
    # Initialize storing space to store all input_dfs
    input_df_list_train = []

    for j in range(len(weather_year_list_training)):
        
        location = locations[i]
        year = weather_year_list_training[j]

        # Get a unique random seed number
        random_seed = random_seeds.iloc[i, j]
        print(random_seed)
        
        # Read NSRDB weather data of the given location of the given year
        NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

        # Prepare weather data file using NSRDB data
        weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lats[i], lons[i], timezones[i])

        # Prepare heating and cooling load using weather data and passive model
        NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lats[i])

        # Prepare solar PV capacity factor using weather data
        pv_cf = Solar_Generation.generate_pv(weather_data, lats[i])

        # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
        load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
        
        # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
        input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)
        
        # Add input_df to list
        input_df_list_train.append(input_df)

    # Run optimization function (Replace with SO)
    PV_Size, Battery_Size, PCM_Heating_Size, PCM_Cooling_Size, ObjValue, First_stage_cost, Second_stage_cost = SO.SO_training(input_df_list_train, Input_Parameters.lossofloadcost)

    # Store the variables in the nested dictionary
    training_results[location] = {
        'PV_Size': PV_Size,
        'Battery_Size': Battery_Size,
        'PCM_Heating_Size': PCM_Heating_Size,
        'PCM_Cooling_Size': PCM_Cooling_Size,
        'Total Cost': ObjValue,
        'Capital Cost': First_stage_cost,
        'Operation Cost': Second_stage_cost
    }



1
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_1998.csv
2
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_1999.csv
3
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2000.csv
4
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2001.csv
5
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2002.csv
6
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2003.csv
7
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2004.csv
8
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2005.csv
9
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2006.csv
10
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2007.csv
11
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2008.csv
12
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2009.csv
13
Reading fi

In [8]:
print(training_results)

{'Arizona': {'PV_Size': 17.899, 'Battery_Size': 18.708, 'PCM_Heating_Size': 57.773, 'PCM_Cooling_Size': 0.0, 'Total Cost': 22978.102, 'Capital Cost': 22877.576, 'Operation Cost': 100.526}, 'Alaska': {'PV_Size': 213.745, 'Battery_Size': 32.656, 'PCM_Heating_Size': 222.485, 'PCM_Cooling_Size': 0.0, 'Total Cost': 44556.382, 'Capital Cost': 43853.028, 'Operation Cost': 703.355}, 'Minnesota': {'PV_Size': 75.416, 'Battery_Size': 16.055, 'PCM_Heating_Size': 196.797, 'PCM_Cooling_Size': 0.0, 'Total Cost': 32105.921, 'Capital Cost': 30662.699, 'Operation Cost': 1443.222}, 'Florida': {'PV_Size': 6.829, 'Battery_Size': 14.676, 'PCM_Heating_Size': 0.0, 'PCM_Cooling_Size': 28.665, 'Total Cost': 20837.661, 'Capital Cost': 20814.977, 'Operation Cost': 22.684}, 'HalfMoonBay': {'PV_Size': 12.52, 'Battery_Size': 6.54, 'PCM_Heating_Size': 59.533, 'PCM_Cooling_Size': 0.0, 'Total Cost': 21412.323, 'Capital Cost': 21161.769, 'Operation Cost': 250.555}}


In [9]:
# Testing:

# Create the nested dictionary
testing_results = {location: {year: {} for year in weather_year_list_testing} for location in locations}

for i in range(len(locations)):
    location = locations[i]
    test_capacities = [training_results[location]['PV_Size'], training_results[location]['Battery_Size'], training_results[location]['PCM_Heating_Size'], training_results[location]['PCM_Cooling_Size']]
    for j in range(len(weather_year_list_testing)):
        year = weather_year_list_testing[j]
        
        # Get a unique random seed number
        random_seed = random_seeds.iloc[i, j+len(weather_year_list_training)]
        print(random_seed)
        
        # Read NSRDB weather data of the given location of the given year
        NSRDB_raw_weather = Data_Conversion.read_NSRDB(data_dir, location, year)

        # Prepare weather data file using NSRDB data
        weather_data = Data_Conversion.prepare_NSRDB(NSRDB_raw_weather, lats[i], lons[i], timezones[i])

        # Prepare heating and cooling load using weather data and passive model
        NetHeatTransfers = Passive_Model.passive_model(Input_Parameters.calibration_file_path, weather_data, Input_Parameters.T_indoor_constant, lats[i])

        # Prepare solar PV capacity factor using weather data
        pv_cf = Solar_Generation.generate_pv(weather_data, lats[i])

        # Prepare occupancy and electrical load schedule using for a specific random seed number for a specific year at a specific location
        load_sched = Electrical_Load.generate_schedules("bayes", weather_data, random_seed)
        
        # Combine all relative input data as input_df, which will be the input of the capacity optimization algorithm
        input_df = Data_Conversion.combine_input_NSRDB(weather_data, load_sched, pv_cf, NetHeatTransfers)
    
        ObjValue, First_stage_cost, Second_stage_cost = SO.simulate(input_df, Input_Parameters.lossofloadcost, test_capacities)
        # Store the variables in the nested dictionary
        testing_results[location][year] = {
            'Total Cost': ObjValue,
            'Capital Cost': First_stage_cost,
            'Operation Cost': Second_stage_cost
        }

21
Reading file for Arizona: Data/NREL_NSRDB_Arizona/323705_33.45_-112.06_2018.csv
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-15
Read LP format model from file /var/folders/ll/cq_mlqg56938kmwh8dys19n80000gn/T/tmpk6bqv6c1.pyomo.lp
Reading time = 0.29 seconds
x1: 157683 rows, 140160 columns, 385434 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads
Optimize a model with 157683 rows, 140160 columns and 385434 nonzeros
Model fingerprint: 0x9ce39549
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+03, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-04, 1e+02]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 92016 rows and 55758 columns
Presolve time: 0.17s
Presolved: 65667 rows, 84402 columns, 206036 nonzeros

Ordering time: 0.01s

Barrier 

In [10]:
# Initialize a dictionary to store averaged results
testing_averaged_results = {}

# Iterate over each location
for location, years_data in testing_results.items():
    # Initialize a dictionary to store sums and counts for averaging
    sums = {
        'Total Cost': 0,
        'Capital Cost': 0,
        'Operation Cost': 0
    }
    count = 0

    # Iterate over each year for the location
    for year, data in years_data.items():
        for key in sums:
            sums[key] += data[key]
        count += 1

    # Calculate averages
    testing_averaged_results[location] = {key: value / count for key, value in sums.items()}

# Print the averaged results
print(testing_averaged_results)

{'Arizona': {'Total Cost': 23208.200999999997, 'Capital Cost': 22877.555, 'Operation Cost': 330.646}, 'Alaska': {'Total Cost': 43853.054, 'Capital Cost': 43853.054, 'Operation Cost': 0.0}, 'Minnesota': {'Total Cost': 31718.2038, 'Capital Cost': 30662.716000000004, 'Operation Cost': 1055.4878}, 'Florida': {'Total Cost': 20814.998, 'Capital Cost': 20814.998, 'Operation Cost': 0.0}, 'HalfMoonBay': {'Total Cost': 21667.8956, 'Capital Cost': 21161.765, 'Operation Cost': 506.13059999999996}}


In [11]:
from pathlib import Path

# Define the output file path
output_file = "Results_SA_Location_SO.xlsx"

# Store Training Result
train_df = pd.DataFrame(training_results).T 
sheet_name_train = "Training Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"File '{output_file}' created with sheet '{sheet_name_train}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        train_df.to_excel(writer, sheet_name=sheet_name_train)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_train}'.")


# Store Testing Result
test_df = pd.DataFrame(testing_averaged_results).T 
sheet_name_test = "Testing Results"

# Check if the file exists
if not Path(output_file).exists():
    # If the file does not exist, create it
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"File '{output_file}' created with sheet '{sheet_name_test}'.")
else:
    # If the file exists, append the new sheet
    with pd.ExcelWriter(output_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        test_df.to_excel(writer, sheet_name=sheet_name_test)
    print(f"Data written to '{output_file}' in sheet '{sheet_name_test}'.")

Data written to 'Results_SA_Location_SO.xlsx' in sheet 'Training Results'.
Data written to 'Results_SA_Location_SO.xlsx' in sheet 'Testing Results'.
